# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [100]:
import numpy as np
import cvxopt

In [101]:
def build_kernel(data_set,d, kernel_type='linear'): #we add the dimension as a variable
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.dot(data_set.T, data_set.T)**d
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [236]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from itertools import combinations


In [248]:
iris = load_iris()
data_set = iris.data
labels = iris.target


In [249]:
def choose_set_for_label(data_set, label,i):

   data_set = data_set[label!=i]
   label = label[label!=i]
  
   train_data_set, test_data_set, train_labels, test_labels = train_test_split(
   data_set, label, test_size=0.2, random_state=15)
   print("Nigga",train_labels)
   min_label = np.min(train_labels)
   max_label = np.max(train_labels)
   train_labels[train_labels == min_label] = -1
   test_labels[test_labels == min_label] = -1
   train_labels[train_labels == max_label] = 1
   test_labels[test_labels == max_label] = 1
   

   return train_data_set, test_data_set, train_labels, test_labels, min_label, max_label

In [250]:
def get_labels_count(data_set):
    
    labels_count = len(data_set)
    
    return labels_count

Use the code that we have implemented earlier:

In [251]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [252]:
from sklearn.metrics import accuracy_score 


predictions=[]  
for i in range(3): #We have three classes:
 pred_aux=[]
 train_data_set, test_data_set, train_labels, test_labels, min_label, max_label = choose_set_for_label(data_set, labels,i)
 objects_count = get_labels_count(train_data_set)
 lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
 predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
 predicted = list(predicted.astype(int))
 
 for m in range(len(predicted)):
        if int(predicted[m]) == -1:
            pred_aux.append(min_label)
        elif int(predicted[m]) == 1:
            pred_aux.append(max_label)            
 predictions.append(pred_aux)
print(predictions)

#We have three lists with the predictions for each classifier. Now we will put the predictions for each data inside
# a list, which will then be subject to a voting:

prediction_final = []
for j in range(len(predictions[0])):
    prediction_split = []
    for k in range(len(predictions)):
      prediction_split.append(predictions[k][j])
    counts = np.bincount(prediction_split)
    prediction_final.append(np.argmax(counts))
    
    
print("Final accuracy:",accuracy_score(prediction_final, test_labels))
#https://cvxopt.org/examples/tutorial/lp.html

Nigga [2 1 1 2 1 2 2 1 1 1 2 2 1 2 1 2 2 2 2 1 2 1 2 2 2 2 1 2 1 2 2 2 1 1 1 1 1
 2 2 2 2 1 1 2 1 2 2 2 1 1 2 1 1 2 2 1 2 2 1 1 1 1 2 1 1 2 1 1 1 1 1 2 2 2
 1 1 1 1 1 2]
     pcost       dcost       gap    pres   dres
 0:  8.4860e+01 -1.3263e+03  3e+03  2e-01  2e-15
 1:  5.7011e+01 -1.1832e+02  2e+02  5e-03  2e-15
 2:  7.1294e+00 -1.6018e+01  2e+01  5e-16  3e-15
 3: -4.3414e-01 -3.6152e+00  3e+00  2e-16  1e-15
 4: -1.1330e+00 -1.7022e+00  6e-01  2e-16  4e-16
 5: -1.2580e+00 -1.4691e+00  2e-01  2e-16  2e-16
 6: -1.3321e+00 -1.4470e+00  1e-01  2e-16  2e-16
 7: -1.3584e+00 -1.3700e+00  1e-02  2e-16  2e-16
 8: -1.3632e+00 -1.3635e+00  3e-04  3e-16  2e-16
 9: -1.3633e+00 -1.3633e+00  5e-06  2e-16  2e-16
10: -1.3633e+00 -1.3633e+00  5e-08  2e-16  2e-16
Optimal solution found.
Nigga [2 0 0 2 0 2 2 0 0 0 2 2 0 2 0 2 2 2 2 0 2 0 2 2 2 2 0 2 0 2 2 2 0 0 0 0 0
 2 2 2 2 0 0 2 0 2 2 2 0 0 2 0 0 2 2 0 2 2 0 0 0 0 2 0 0 2 0 0 0 0 0 2 2 2
 0 0 0 0 0 2]
     pcost       dcost       gap    pres   dres
 

I know the code is wrong, but I cannot make it work. 

I know for sure that the error is in how I am approaching the exercise. The code you can see right now considers that the test set is the same for the three classifiers, which is wrong. From this point, I can understand that the code will not work. To be honest, I do not understand how to proceed with the voting with multiple classes.